In [1]:
from sklearn.datasets import fetch_20newsgroups
train=fetch_20newsgroups(subset='train')
test=fetch_20newsgroups(subset='test')

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(min_df=0.01,max_df=0.1)

train_X=cv.fit_transform(train['data']).toarray()
test_X=cv.transform(test['data']).toarray()

- ここまででbowの特徴ができた。
    - 重ねて言うが基本はsparse matrixで持ってたほうがいい
    - それか頻度でフィルタしてちゃんと単語を減らしておくか
        - 50000単語くらいまでが妥当(だと勝手に思っている)
- 大事なこと
    - trainとtest、説明変数と目的変数は絶対に別の変数に入れる
    - なんならファイルも分けておくべき
    - そうしないと死を迎えてしまう
        - 実際迎えた。

In [3]:
train_X.shape

(11314, 1983)

- 今回は1983種類の単語しか使ってないっぽい
    - 多分もうちょい使ったほうが精度的には良い
    - でも今回はsurfaceで動かしたいだけだから(めんどいから)チューニングはパス

In [4]:
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [5]:
clf_list=[RidgeClassifier(),RandomForestClassifier(n_estimators=100),SVC(),SVC(kernel='poly',degree=2)]
clf_names=['L2正則化つきの線形分類器','ランダムフォレスト','rbfカーネルのSVM','2次の多項カーネルのSVM']

for name,clf in zip(clf_names,clf_list):
    clf.fit(train_X,train['target'])
    print(name,'：',clf.score(test_X,test['target']))

L2正則化つきの線形分類器 ： 0.629978757302
ランダムフォレスト ： 0.699548592671
rbfカーネルのSVM ： 0.502389803505
2次の多項カーネルのSVM ： 0.0677110993096


- 手法ごとに計算時間が違う
    - 次元とデータ数に気を払うべき
    - 10万データとかあるときにSVMを使うと死を迎えてしまう
        - 実際迎えた。
        - 1万データで回している今でさえも後悔している。
        - SVMはだいたいデータ量の3乗オーダだと言われている。厳密には違うらしい。

- さっきのナイーブベイズは0.46123207647371217(参考記録)。
    - パラメータチューニングとかデータの相性とかもあるので一概に言えない。
    - 今回のデータの作り方だと2次の多項カーネルが爆死している。
        - 多分低次元だから。
        - チャンスレートはだいたい1/20(=0.05)なので、ほとんど当てずっぽう状態だとわかる。
        - 余談だが、グラフを描いて精度の良し悪しを言いたいときはチャンスレートを直線で引くと丁寧らしい。
    - ちゃんとデータの性質を見極めよう。